In [14]:
import os
from google.cloud import speech_v1
from google.cloud.speech_v1 import enums
from google.datalab import storage

In [20]:
#os.environ["GOOGLE_APPLICATION_CREDENTIALS"]="/Users/clairepajot/Downloads/speech-rec-key.json"
os.environ["GOOGLE_APPLICATION_CREDENTIALS"]="speech-rec-key.json"

In [16]:
URI_ROOT = "gs://aave-test-bucket"
AUDIO_FILES = [o.key for o in storage.Bucket('aave-test-bucket').objects() if o.key.startswith('DCA_audio_part01_2018.10.06')]

In [17]:
def get_single_transcript(storage_uri):
    """
    Transcribe long audio file from Cloud Storage using asynchronous speech
    recognition

    Args:
      storage_uri URI for audio file in Cloud Storage, e.g. gs://[BUCKET]/[FILE]
    """

    client = speech_v1.SpeechClient()

    # The language of the supplied audio
    language_code = "en-US"

    config = {
        "language_code": language_code,
    }
    
    audio = {"uri": storage_uri}

    operation = client.long_running_recognize(config, audio)

    print(u"Waiting for operation to complete...")
    response = operation.result()
    
    transcript = []
    for result in response.results:
        # First alternative is the most probable result
        alternative = result.alternatives[0]
        transcript.append(alternative.transcript.strip())
    
    return " ".join(transcript)

In [19]:
for audio_file in AUDIO_FILES:
    full_file_path = os.path.join(URI_ROOT, audio_file)
    transcript = get_single_transcript(full_file_path)
    #To do: Any pre-processing on the transcript
    print(transcript)

Waiting for operation to complete...
count to 10 what's your name can you count to 10 for me please 1 2 3 4 5 6 7 8 9 10 what kind of games do you play around here football jump rope that's all what do you do when you jump rope who played double dutch you have to tell people how to be how to turn in vote remind me to Jumping about to start the job do you have any rhymes no Giver play games where you hide or doing work person got a count to 20 and why do some more children have you got time to find how do you pick a person I don't know new plus-size sunlight and somebody said that they're going to be the name and then just sound like South Africa I played don't I have I have some bosses games at home don't spill the beans and I have a puzzle and I have a game called eggnog and four dices supposed to roll up roll the dice and have some holes in it whatever you want to die stop and supposed to put the marbles in the hole and then how do you win it yeah for things down the road and then yo